## This notebook takes in a trained model (and its relevant data) and performs inference on a given screening library. At the end we output compounds from the screening library that are predicted to have a higher potency than the top few known actives.

In [1]:
import multiprocessing as mp
import os
import pickle
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
from functools import partial
from tqdm import tqdm
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
import pdb
import time
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from fastai.basics import *
from fastai.tabular import *

In [2]:
def Morgan_Fingerprint(smile):
    return AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(smile),3,nBits=512, useFeatures = True)

def Atom_Pair(smile):
    return rdMolDescriptors.GetHashedAtomPairFingerprintAsBitVect(Chem.MolFromSmiles(smile),nBits = 512)

def TopologicalTorsion(smile):
    return rdMolDescriptors.GetHashedTopologicalTorsionFingerprintAsBitVect(Chem.MolFromSmiles(smile),nBits = 512)

def fingerprints(smile):
    MF = Morgan_Fingerprint(smile)
    AP = Atom_Pair(smile)
    TT = TopologicalTorsion(smile)
    return np.array(MF + AP + TT)

### What model to use? What screening library for inference shall we use?

In [3]:
main_dir = 'Model_Build/' # contains trained models and the relevant underlying data, both processed and raw
model_file = 'noncovalent_model.pkl' # trained models
model_data = 'noncovalent_model_data' # processed data
raw_data = 'known_noncovalent_activity.csv' # raw data

##############

screening_library_folder_dir = 'Screening_Library/noncovalent_library' # directory where the screening libraries are stored

### Here we pull in the original training data in order to apply the same PCA transforms to the new inference data.

In [4]:
with open(main_dir + model_data, 'rb') as filehandle:
    X_train = pickle.load(filehandle)[0]

preprocess = PCA(n_components = 20).fit_transform(X_train)

### Pull in the pre-saved FASTAI model and make predictions -- [ Lower_Activity, Higher_Activity ] are the classes here. So a prediction of '1' means first compound is predicted to have higher activity than the second compound.

In [ ]:
# Apply PCA to new inference data and then generated the required dataframe to pass to FASTAI model
def fastai_preprocess(X, pca_preprocess):
    X = pca_preprocess.transform(X)
    columns = ['Feature ' + str(i) for i in range(X.shape[1])]
    return pd.DataFrame(X, columns = columns)

# Load pre-saved FASTAI model and run predictions...return tuple of probabilities as well as predicted class
def fastai_inference(df_test):
    learner = load_learner(main_dir, model_file, test = TabularList.from_df(df_test))
    probs,_ = learner.get_preds(ds_type=DatasetType.Test)
    return probs.numpy(), probs.argmax(dim = -1).numpy()

# This provides a sense check that predicedt probabilities are correlated across the various known actives compounds.
def prediction_correlation(probs_a, probs_b):
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Higher_Activity_than_compound_b', fontsize = 15)
    ax.set_ylabel('Higher_Activity_than_compound_a', fontsize = 15)
    ax.scatter(probs_b, probs_a, c = 'r', alpha=0.6)
    fig.suptitle('Correlation: ' + str(np.round(100 * pearsonr(probs_a, probs_b)[0],1)) + '%', fontsize=14)
    ax.plot()

# Create an inference dataframe of length j x k where j is number of screening compounds and k is the number of known actives
def inference_df(active_smiles, active_fps, candidate_smiles, candidate_fps):
    all_df = []

    for index, active in enumerate(active_fps):
        X = candidate_fps - np.array(active) ## ALWAYS SCREENING COMPOUND - KNOWN ACTIVE COMPOUND
        X_df = fastai_preprocess(X, preprocess) # now let's convert to a dataframe

        probs, preds = fastai_inference(X_df)
        more_active = preds == 1
        print('Known Active ' + str(index + 1 ), ' : % predicted to be more potent than this active: ', str(100 * more_active.sum()/len(more_active)))

        # Now build a nice dataframe
        df = pd.DataFrame(data = candidate_smiles, columns = ['Candidate_SMILES'])
        df['Known_Active_SMILES'] = active_smiles[index]
        df['Higher_Activity'] = preds
        df['Prob_Higher_Activity'] = list(zip(*probs))[1]
        all_df.append(df)

    return all_df

### Let's find the best set actives we have so far to act as the 'benchmark' for inference

In [ ]:
known_data = pd.read_csv(main_dir + raw_data).sort_values(by = 'IC50', ascending = True).reset_index(drop = True)
active_smiles = known_data['SMILES'].to_list()[0:4]
active_fps = [np.array(fingerprints(smi)) for smi in active_smiles]

## Now here are 2 ways to do inference. All produce 'final_df'; the dataframe of all predictions. The input screening library must contain 'SMILES' as a column.

### Option 1: Multiple libraries in a folder -- assumes Fingerprints are pre-computed

In [ ]:
final_dfs = []
path = screening_library_folder_dir
for file in tqdm(os.listdir(path)):
    if 'Section' in file: # PostEra's preprocessing of large screening libraries broken up into many compressed CSVs labelled Section_X.pkl
        with open(path + file, 'rb') as filehandle:
            section_data = pd.read_pickle(path + file)
        candidate_smiles, candidate_fps = section_data['SMILES'].to_list(), np.vstack(section_data['fingerprints'])
        final_dfs.append(inference_df(active_smiles, active_fps, candidate_smiles, candidate_fps))
final_df = [ pd.concat(df, ignore_index = True) for df in final_dfs]
final_df = pd.concat(final_df, ignore_index = True)

### Option 2: A single library. Will compute Fingerprints on the fly.

In [ ]:
# single_library_file = 'model_time_split_data.csv'
# section_data = pd.read_csv(single_library_file).drop_duplicates(subset = ['SMILES']).reset_index(drop = True).fillna(np.inf)
# section_data['fingerprints'] = [fingerprints(row['SMILES']) for index, row in section_data.iterrows()]
# candidate_smiles, candidate_fps = section_data['SMILES'].to_list(), np.vstack(section_data['fingerprints'])

# final_dfs = inference_df(active_smiles, active_fps, candidate_smiles, candidate_fps)
# final_df = pd.concat(final_dfs, ignore_index = True).reset_index(drop = True)

### Add 2 columns if doing time-split dataset to see which compounds the model correctly identified as having higher IC50 than known top actives

In [ ]:
# final_df['Candidate_IC50'] = np.tile(section_data['IC50'].to_numpy(), len(active_smiles))
# final_df['Known_Active_IC50'] = np.repeat(known_data.head(5)['IC50'].to_numpy(), section_data.shape[0])

### Sense-Check here: Analyse correlation amongst predictions. Basically for 2 known actives we look at the correlation of their probabilities of being lower/higher potency than the screening compounds. Intuitively the correlation should be very high meaning that the candidate compounds are ranked consistently.

In [ ]:
compound_a, compound_b = 2,3
rand_section = random.randint(0, len(final_dfs)-1)
probs_a, probs_b = final_dfs[rand_section][compound_a]['Prob_Higher_Activity'], final_dfs[rand_section][compound_b]['Prob_Higher_Activity']

prediction_correlation(probs_a, probs_b)

### Let's take all the results (final_df) and filter to find the compounds predicted to have a higher activity than all our baseline actives. Then also pivot the data to show a nice table of these new exciting compounds compared with known actives.

In [ ]:
min_probs = final_df.groupby(by = 'Candidate_SMILES')['Prob_Higher_Activity'].min()
min_hits = min_probs[min_probs > 0.5].to_frame().reset_index(level = 'Candidate_SMILES').sort_values(by = 'Prob_Higher_Activity', ascending = False)
print('Number of hits that are predicted to have higher activity than all top actives: ', min_hits.shape[0])

In [ ]:
pivoted = final_df.drop_duplicates(subset = ['Candidate_SMILES', 'Known_Active_SMILES']).pivot(index = 'Candidate_SMILES', columns = 'Known_Active_SMILES', values = 'Prob_Higher_Activity')
pivoted['Min_Probability'] = [ row.min() for index, row in pivoted.iterrows() ]
pivoted = pivoted[pivoted['Min_Probability'] > 0.5]
new_hits = pivoted.sort_values(by = ['Min_Probability'], ascending = False)

### Output the promising compounds somewhere

In [ ]:
output_dir = 'noncovalent_top_ranked_compounds.csv'
new_hits.to_csv(output_dir)